<a href="https://colab.research.google.com/github/GummyBear-w/aop113b/blob/main/EX04_03_%E5%AE%A2%E6%88%B6%E5%88%86%E7%BE%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 問題定義

將問題欄位 label encoding  
將客戶分成3群，並解讀各群特質  
計算 k = 2 ~ 15 的 Calinski-Harbasz Score，找出最佳 k 值

## 資料收集

In [39]:
!wget -O car_models.csv https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv

--2025-05-30 08:21:00--  https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2839 (2.8K) [text/plain]
Saving to: ‘car_models.csv’

car_models.csv      100%[===================>]   2.77K  --.-KB/s    in 0s      

2025-05-30 08:21:00 (30.0 MB/s) - ‘car_models.csv’ saved [2839/2839]



In [40]:
import pandas as pd
url = 'https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv'
df = pd. read_csv(url)
df.head()

,性別,年齡,收入（千）,消費指數（1~100）
0,女,74,38,81
1,女,51,71,91
2,女,30,65,10
3,女,88,49,17
4,女,55,48,70


## 資料前處理

### 資料清理

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   性別           200 non-null    object
 1   年齡           200 non-null    int64 
 2   收入（千）        200 non-null    int64 
 3   消費指數（1~100）  200 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 6.4+ KB


### 探索性分析

In [42]:
df_cor = df.drop(columns='性別').corr()
df_cor

,年齡,收入（千）,消費指數（1~100）
年齡,1.000000,0.031519,-0.127454
收入（千）,0.031519,1.000000,0.031476
消費指數（1~100）,-0.127454,0.031476,1.000000


### 資料分割

類別轉換

In [43]:
df['性別'].unique ()

array(['女', '男'], dtype=object)

In [44]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder (categories=[['女','男']])
df[['性別']] = encoder.fit_transform(df[['性別']])
df

,性別,年齡,收入（千）,消費指數（1~100）
0,0.0,74,38,81
1,0.0,51,71,91
2,0.0,30,65,10
3,0.0,88,49,17
4,0.0,55,48,70
...,...,...,...,...
195,1.0,86,84,82
196,1.0,59,52,30
197,0.0,63,29,61
198,1.0,67,80,9


## 模型訓練

In [45]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)
kmeans.fit(df)

KMeans(n_clusters=3)

In [46]:
df['cluster'] = kmeans.labels_
df

,性別,年齡,收入（千）,消費指數（1~100）,cluster
0,0.0,74,38,81,1
1,0.0,51,71,91,0
2,0.0,30,65,10,2
3,0.0,88,49,17,2
4,0.0,55,48,70,1
...,...,...,...,...,...
195,1.0,86,84,82,0
196,1.0,59,52,30,2
197,0.0,63,29,61,1
198,1.0,67,80,9,2


## 模型評估

In [47]:
df. groupby('cluster').mean()

,性別,年齡,收入（千）,消費指數（1~100）
cluster,,,,
0,0.636364,49.254545,79.927273,77.945455
1,0.492537,50.955224,38.686567,69.298507
2,0.589744,56.910256,61.243590,20.282051


In [48]:
from sklearn.metrics import calinski_harabasz_score
score = calinski_harabasz_score(df.drop(columns='cluster'), kmeans. labels_)
score

np.float64(97.22455803987141)

## 模型調整

In [49]:
df_nocluster = df.drop(columns='cluster')
for i in range (2,16):
  kmeans = KMeans(n_clusters=i)
  kmeans. fit(df_nocluster)
  score = calinski_harabasz_score(df_nocluster, kmeans.labels_)
  print(f'k={i} score={score}')

k=2 score=111.68583137120146
k=3 score=95.36808392176823
k=4 score=90.52713165468991
k=5 score=89.64677015045184
k=6 score=74.51064950390621
k=7 score=93.21431213456341
k=8 score=91.25339971424495
k=9 score=97.0210438668853
k=10 score=92.23894788405498
k=11 score=88.74197178320308
k=12 score=88.31695267289375
k=13 score=88.48815162168059
k=14 score=80.54182268482717
k=15 score=79.24852897386477
